# Clustering 101

## What is clustering?

> Clustering is a Machine Learning technique that involves the grouping of data points. Given a set of data points, we can use a clustering algorithm to classify each data point into a specific group. In theory, data points that are in the same group should have similar properties and/or features, while data points in different groups should have highly dissimilar properties and/or features. Clustering is a method of unsupervised learning and is a common technique for statistical data analysis used in many fields.
 - https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68

## K-Means

### Getting Started

This interactive exploration of the k-means algorithm relies on a few Python packages:

- `numpy`
- `scikit-learn`
- `ipywidgets`
- `bqplot`

Go ahead and install any you're missing before proceeding.

In [1]:
import time
import numpy as np

import bqplot.pyplot as plt
from bqplot import *

from ipywidgets import HBox, VBox, IntSlider, Play, jslink, Button

from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans

np.random.seed(0)

### Data
We can rely on scikit-learn's built in `make_blobs` function to generate some sample clustering data to experiment on.

In [2]:
data, actual = make_blobs(n_samples=300, centers=3, n_features=2, random_state=0, cluster_std=0.5)
assigned_cluster = [0] * 300

This produces data that is already nicely grouped. Starting with clean data like this can help us visualize how the algorithm works.

In [3]:
sc_x = LinearScale()
sc_y = LinearScale()

c_ord = OrdinalColorScale(colors=['DodgerBlue', 'SeaGreen', 'OrangeRed', 'Yellow', 'HotPink', 'Black', 'Purple',
                                  'orange', 'red', 'blue'])

initial_data_plot = Scatter(x=data[:, 0], y=data[:, 1],
                color=actual,
                scales={'x': sc_x, 'y': sc_y, 'color': c_ord}, 
                default_size=10)

ax_x = Axis(scale=sc_x, label='x')
ax_y = Axis(scale=sc_y, orientation='vertical', label='y')
ax_c = ColorAxis(scale=c_ord, label='Class', side='right', orientation='vertical')

initial_fig = Figure(marks=[initial_data_plot], axes=[ax_x, ax_y, ax_c])
initial_fig

Figure(axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='vertical', scale=LinearScale()…

### The Algorithm



#### Step 0: Initialize cluster centers

We'll explore the details of this process later on, but for now, we'll pick out *k* data points from the dataset at random to serve as the initial centroid for each of our *k* clusters.

In [4]:
init_centroids = np.random.randint(0, 299, size=3)
init_centroids = data[init_centroids]

In [5]:
initial_centroid_plot = Scatter(x=init_centroids[:, 0], y=init_centroids[:, 1],
                        colors=['Yellow'], marker='cross', 
                        stroke='black', stroke_width=0.5,
                        scales={'x': sc_x, 'y': sc_y})

no_assign_plot = Scatter(x=data[:, 0], y=data[:, 1],
                color=[0] * 300,
                scales={'x': sc_x, 'y': sc_y, 'color': c_ord}, 
                default_size=10)

step_0_fig = Figure(marks=[no_assign_plot, initial_centroid_plot], axes=[ax_x, ax_y, ax_c])
step_0_fig

Figure(axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='vertical', scale=LinearScale()…

#### Step 1: Assign each data point to a centroid

This is the first of our two iterative steps. Start by computing the Euclidean distance (i.e. $d(p, q) = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + ... + (q_n - p_n)^2}$ ) from each data point to each of the centroids. We then 'assign' each data point to the cluster whose centroid is the closest.

In [6]:
def assign_centroid(data, centroids):
    distances = pairwise_distances(data, centroids, metric='euclidean')
    assignments = np.argmin(distances, axis=1)
    
    return assignments

In [7]:
assigned_cluster = assign_centroid(data, init_centroids)

data_plot = Scatter(x=data[:, 0], y=data[:, 1],
                color=assigned_cluster,
                scales={'x': sc_x, 'y': sc_y, 'color': c_ord}, 
                default_size=10)

step_1_fig = Figure(marks=[data_plot, initial_centroid_plot], axes=[ax_x, ax_y, ax_c])
step_1_fig

Figure(axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='vertical', scale=LinearScale()…

#### Step 2: Revise centroids

The second of the iterative steps is to update the location of the centroids. The mean of the data points assigned to each cluster becomes the new centroid.

In [8]:
def revise_centroids(data, num, assignments):
    new_centroids = []
    
    for i in range(num):
        member_data = data[assignments == i, :]
        centroid = member_data.mean(axis=0)
        new_centroids.append(centroid)

    return np.array(new_centroids)

In [9]:
centroids = revise_centroids(data, 3, assigned_cluster)

updated_centroid_plot = Scatter(x=centroids[:, 0], y=centroids[:, 1],
                        colors=['Yellow'], marker='cross', 
                        stroke='black', stroke_width=0.5,
                        scales={'x': sc_x, 'y': sc_y})

step_2_fig = Figure(marks=[data_plot, updated_centroid_plot], axes=[ax_x, ax_y, ax_c])
step_2_fig

Figure(axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='vertical', scale=LinearScale()…

#### Step 3: Repeat

We will repeat steps 1 and 2 until the algorithm has converged. Convergence is achieved when the cluster assignments for each data point no longer change between iterations. Convergence of the k-means algorithm does not necessarily indicate the optimum solution.

#### All together

In [10]:
time_interval = 3000

step_slider = IntSlider(min=0, max=7, step=1, description='Step', value=0)

anim_centroids = init_centroids
anim_assignments = [0]*300

anim_data_plot = Scatter(x=data[:, 0], y=data[:, 1],
                color=[0]*300,
                scales={'x': sc_x, 'y': sc_y, 'color': c_ord}, 
                default_size=10)

anim_centroid_plot = Scatter(x=anim_centroids[:, 0], y=anim_centroids[:, 1],
                        colors=['Yellow'], marker='cross', 
                        stroke='black', stroke_width=0.5,
                        scales={'x': sc_x, 'y': sc_y})

anim_label = Label(x=[0.1], y=[0.1], default_size=30, font_weight='bolder', colors=['orange'],
                   text=['Data'], enable_move=True)

anim_fig = Figure(marks=[anim_data_plot, anim_label], axes=[ax_x, ax_y, ax_c], animation_duration=int(time_interval*0.6),
                  title='k-means (k=3)', min_aspect_ratio=1, max_aspect_ratio=1)


def step_changed(change):
    global anim_data_plot
    global anim_fig
    global anim_assignments
    global anim_centroids
    
    val = step_slider.value
    
    if val == 0:    
        anim_data_plot.color = [0]*300
        anim_fig.marks = [anim_data_plot, anim_label]

        anim_label.text = ['Data']
    elif val == 1:
        anim_data_plot.color = [0]*300    
        anim_centroids = init_centroids

        anim_centroid_plot.x = anim_centroids[:, 0]
        anim_centroid_plot.y = anim_centroids[:, 1]

        anim_fig.marks = [anim_data_plot, anim_label, anim_centroid_plot]

        anim_label.text = ['Initialize centroids']
    elif val == 2:
        anim_centroids = init_centroids
    
        anim_assignments = assign_centroid(data, anim_centroids)
        anim_data_plot.color = anim_assignments

        anim_label.text = ['Assign clusters']
    elif val in [3, 5, 7]:
        anim_centroids = revise_centroids(data, 3, anim_assignments)
    
        anim_centroid_plot.x = anim_centroids[:, 0]
        anim_centroid_plot.y = anim_centroids[:, 1]

        anim_label.text = ['Update centroids']
    elif val in [4, 6]:
        anim_assignments = assign_centroid(data, anim_centroids)
        anim_data_plot.color = anim_assignments

        anim_label.text = ['Assign clusters']

step_slider.observe(step_changed, 'value')

play_button = Play(min=0, max=7, interval=time_interval)
jslink((play_button, 'value'), (step_slider, 'value'))        

In [11]:
VBox([HBox([play_button, step_slider]), anim_fig])

### Initial Centroid Selection

A key consideration when using k-means is the importance of the centroids we initialize the algorithm with. Not only will different initial centroids impact the number of iterations the algorithm will take to converge, it will also impact the quality of the clusters obtained at convergence.

We can experiment with the impact of this selection by dragging the initial centroid positions in the following figure.

In [12]:
sel_centroids = init_centroids
sel_assignments = [0]*300

sel_data_plot = Scatter(x=data[:, 0], y=data[:, 1],
                    color=[0]*300,
                    scales={'x': sc_x, 'y': sc_y, 'color': c_ord}, 
                    default_size=10)

sel_centroid_plot = Scatter(x=sel_centroids[:, 0], y=sel_centroids[:, 1],
                            colors=['Yellow'], marker='cross', 
                            stroke='black', stroke_width=0.5,
                            scales={'x': sc_x, 'y': sc_y},
                            enable_move=True)

sel_label_1 = Label(x=[0.1], y=[0.2], default_size=12, font_weight='bolder', colors=['orange'],
                       text=['0: {}'.format(sel_centroids[0])], enable_move=True)

sel_label_2 = Label(x=[0.1], y=[0.15], default_size=12, font_weight='bolder', colors=['orange'],
                       text=['1: {}'.format(sel_centroids[1])], enable_move=True)

sel_label_3 = Label(x=[0.1], y=[0.1], default_size=12, font_weight='bolder', colors=['orange'],
                       text=['2: {}'.format(sel_centroids[2])], enable_move=True)

sel_fig = Figure(marks=[sel_data_plot, sel_centroid_plot, sel_label_1, sel_label_2, sel_label_3], 
                     axes=[ax_x, ax_y, ax_c], animation_duration=1000,
                      title='k-means (k=3)', min_aspect_ratio=1, max_aspect_ratio=1)


def update_cent(change=None):
    sel_centroids[:, 0] = sel_centroid_plot.x
    sel_centroids[:, 1] = sel_centroid_plot.y
    
    sel_label_1.text = ['0: {}'.format(sel_centroids[0])]
    sel_label_2.text = ['1: {}'.format(sel_centroids[1])]
    sel_label_3.text = ['2: {}'.format(sel_centroids[2])]
    
sel_centroid_plot.observe(update_cent, names=['x'])
sel_centroid_plot.observe(update_cent, names=['y'])

In [13]:
start_button = Button(description='Start')
reset_button = Button(description='Reset', disabled=True)

def reset(change):
    global sel_data_plot
    global sel_fig
    global sel_assignments
    global sel_centroids
    
    sel_centroids = init_centroids
    sel_assignments = [0]*300

    sel_data_plot.color = [0]*300
    sel_centroid_plot.x = sel_centroids[:, 0]
    sel_centroid_plot.y = sel_centroids[:, 1]
    
    sel_fig.marks = [sel_data_plot, sel_centroid_plot, sel_label_1, sel_label_2, sel_label_3]
    
    reset_button.disabled = True
    start_button.disabled = False
    sel_centroid_plot.enable_move = True

def start(change):
    global sel_data_plot
    global sel_fig
    global sel_assignments
    global sel_centroids
    
    start_button.disabled = True
    
    sel_centroid_plot.enable_move = False    
    prev_assignments = None
    
    sel_label_4 = Label(x=[0.1], y=[0.15], default_size=18, font_weight='bolder', colors=['orange'], enable_move=True)
    
    sel_fig.marks = [sel_data_plot, sel_centroid_plot, sel_label_4]
    
    for itr in range(100):
        sel_label_4.text = ['Iteration {}'.format(itr+1)]
        
        sel_assignments = assign_centroid(data, sel_centroids)
        sel_data_plot.color = sel_assignments
        
        sel_centroids = revise_centroids(data, 3, sel_assignments)        
        sel_centroid_plot.x = sel_centroids[:, 0]
        sel_centroid_plot.y = sel_centroids[:, 1]
        
        if prev_assignments is not None and (prev_assignments == sel_assignments).all():
            break
            
        prev_assignments = sel_assignments[:]
        time.sleep(1.2)
        
    reset_button.disabled = False
        
start_button.on_click(start)
reset_button.on_click(reset)

In [14]:
VBox([sel_fig, HBox([start_button, reset_button])])

### K-means++

There have been many studies done on algorithms to select the appropriate initial clusters for k-means. We'll explore one such method that is known to generally perform well in balancing computational efficiency and clustering result.

In [15]:
time_interval = 3000
sc_opacity = LinearScale()

km_step_slider = IntSlider(min=0, max=6, step=1, description='Step', value=0)

km_centroids = np.array([])

km_data_plot = Scatter(x=data[:, 0], y=data[:, 1],
                color=[0]*300,
                scales={'x': sc_x, 'y': sc_y, 'color': c_ord, 'opacity': sc_opacity}, 
                default_size=10)

km_centroid_plot = Scatter(x=None, y=None,
                        colors=['Yellow'], marker='cross', 
                        stroke='black', stroke_width=0.5,
                        scales={'x': sc_x, 'y': sc_y})

km_label = Label(x=[0.1], y=[0.1], default_size=18, font_weight='bolder', colors=['orange'],
                   text=['Data'], enable_move=True)

km_fig = Figure(marks=[km_data_plot, km_label], axes=[ax_x, ax_y, ax_c], animation_duration=int(time_interval*0.6),
                  title='k-means (k=3)', min_aspect_ratio=1, max_aspect_ratio=1)


def km_step_changed(change):
    global km_data_plot
    global km_fig
    global km_assignments
    global km_centroids
    
    val = km_step_slider.value
    distances = []
    
    if val == 0:    
        km_centroids = np.array([])
        km_data_plot.color = [0]*300
        km_fig.marks = [km_data_plot, km_label]

        anim_label.text = ['Data']
    elif val == 1:
        point = np.random.randint(0, 299, size=1)
        point = data[point]
        km_centroids = point
        
        km_data_plot.color = [0]*300

        km_centroid_plot.x = km_centroids[:, 0]
        km_centroid_plot.y = km_centroids[:, 1]

        km_fig.marks = [km_data_plot, km_label, km_centroid_plot]

        km_label.text = ['Randomly select point']
    elif val == 2:
        distances = pairwise_distances(data, km_centroids[0:1], metric='euclidean').flatten()**2
        weight = distances/sum(distances)
    
        km_data_plot.default_opacity = weight
    elif val in [3, 5, 7]:
        anim_centroids = revise_centroids(data, 3, anim_assignments)
    
        anim_centroid_plot.x = anim_centroids[:, 0]
        anim_centroid_plot.y = anim_centroids[:, 1]

        anim_label.text = ['Update centroids']
    elif val in [4, 6]:
        anim_assignments = assign_centroid(data, anim_centroids)
        anim_data_plot.color = anim_assignments

        anim_label.text = ['Assign clusters']

km_step_slider.observe(km_step_changed, 'value')

km_play_button = Play(min=0, max=6, interval=time_interval)
jslink((km_play_button, 'value'), (km_step_slider, 'value'))        

In [16]:
VBox([HBox([km_play_button, km_step_slider]), km_fig])

### How many clusters?

In our case, the number of clusters for our k-means model to create is visually obvious, but for most real-world clustering tasks, this will not be true. Let's investigate how we can systematically determine the number of clusters to pick for a dataset.

#### Scikit-Learn

We can utilize scikit-learn's built in KMeans algorithm.

In [17]:
kmeans = []
heterogeneity = []

for i in range(2, 11):
    kmeans.append(KMeans(n_clusters=i, init='k-means++',
                         random_state=0, n_jobs=-1).fit(data))
    
    heterogeneity.append(kmeans[i-2].inertia_)

In [19]:
sc_x_2 = LinearScale()
sc_y_2 = LinearScale()

ax_x_2 = Axis(scale=sc_x_2, label='k')
ax_y_2 = Axis(scale=sc_y, orientation='vertical', label='Heterogeneity')

hetero_plot = Scatter(x=[2, 3, 4, 5, 6, 7, 8, 9, 10],
                      y=heterogeneity,
                      colors=['DodgerBlue'],
                      scales={'x': sc_x_2, 'y': sc_y_2},
                      interactions={'click': 'select'},
                      selected_style={'opacity': 1.0, 'stroke': 'Red', 'fill': 'DarkOrange'},
                      unselected_style={'opacity': 0.5})

hetero_fig = Figure(marks=[hetero_plot], axes=[ax_x_2, ax_y_2],
                       min_aspect_ratio=1, max_aspect_ratio=1,
                       title='Heterogeneity vs. k')

hetero_plot.selected = [1]

hetero_data_plot = Scatter(x=data[:, 0], y=data[:, 1],
                    color=kmeans[hetero_plot.selected[0]].labels_,
                    scales={'x': sc_x, 'y': sc_y, 'color': c_ord}, 
                    default_size=10)

hetero_centroid_plot = Scatter(x=None, 
                               y=None,
                            colors=['Yellow'], marker='cross', 
                            stroke='black', stroke_width=0.5,
                            scales={'x': sc_x, 'y': sc_y})

def on_select(self, target):
    hetero_data_plot.color = [0]*300 if hetero_plot.selected == None else kmeans[hetero_plot.selected[0]].labels_
    hetero_centroid_plot.x = None if hetero_plot.selected == None else kmeans[hetero_plot.selected[0]].cluster_centers_[:, 0]
    hetero_centroid_plot.x = None if hetero_plot.selected == None else kmeans[hetero_plot.selected[0]].cluster_centers_[:, 1]

    
hetero_plot.on_element_click(on_select)

hetero_data_fig = Figure(marks=[hetero_data_plot, hetero_centroid_plot], axes=[ax_x, ax_y, ax_c],
                       min_aspect_ratio=1, max_aspect_ratio=1,
                       title='Data', animation_duration=100)

HBox([hetero_fig, hetero_data_fig])